In [1]:
import os
import zipfile
import mp
from datetime import datetime
from glob import glob
from tqdm import tqdm
from huggingface_hub import HfApi
from huggingface_hub import HfFileSystem
import time

partition_size = 3e+9

In [2]:
!du -hs dedup-parliament

25G	dedup-parliament


In [3]:
files = glob('dedup-parliament/*.mp3')
len(files)

610804

In [4]:
def loop(files):
    files, index = files
    current_index = 0
    api = HfApi()
    fs = HfFileSystem()
    total = 0
    temp = []
    for i in tqdm(range(len(files))):
        s = os.stat(files[i]).st_size
        if s + total >= partition_size:
            part_name = f"dedup-parliament-{index}-{current_index}.zip"
                
            with zipfile.ZipFile(part_name, 'w', zipfile.ZIP_DEFLATED) as zipf:
                for f in temp:
                    zipf.write(f, arcname=f)

            while True:
                try:
                    api.upload_file(
                        path_or_fileobj=part_name,
                        path_in_repo=part_name,
                        repo_id="malaysia-ai/dedup-Malaysian-Emilia",
                        repo_type="dataset",
                    )
                    break
                except:
                    time.sleep(60)

            os.remove(part_name)
            
            current_index += 1
            temp = [files[i]]
            total = s
        else:
            temp.append(files[i])
            total += s
        
    if len(temp):
        part_name = f"dedup-parliament-{index}-{current_index}.zip"

        with zipfile.ZipFile(part_name, 'w', zipfile.ZIP_DEFLATED) as zipf:
            for f in temp:
                zipf.write(f, arcname=f)

        while True:
            try:
                api.upload_file(
                    path_or_fileobj=part_name,
                    path_in_repo=part_name,
                    repo_id="malaysia-ai/dedup-Malaysian-Emilia",
                    repo_type="dataset",
                )
                break
            except:
                time.sleep(60)

        os.remove(part_name)

In [5]:
loop((files[:1000], 0))

100%|█████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 240692.30it/s]


dedup-parliament-0-0.zip:   0%|          | 0.00/41.4M [00:00<?, ?B/s]

In [6]:
mp.multiprocessing(files, loop, cores = 10, returned = False)

100%|███████████████████████████████████████████████████████████████████████████████| 61080/61080 [00:00<00:00, 145533.80it/s]


dedup-parliament-5-0.zip:   0%|          | 0.00/2.50G [00:00<?, ?B/s]

dedup-parliament-2-0.zip:   0%|          | 0.00/2.51G [00:00<?, ?B/s]

dedup-parliament-1-0.zip:   0%|          | 0.00/2.50G [00:00<?, ?B/s]

dedup-parliament-7-0.zip:   0%|          | 0.00/2.51G [00:00<?, ?B/s]

...

dedup-parliament-0-0.zip:   0%|          | 0.00/2.50G [00:00<?, ?B/s]

dedup-parliament-3-0.zip:   0%|          | 0.00/2.51G [00:00<?, ?B/s]

dedup-parliament-4-0.zip:   0%|          | 0.00/2.51G [00:00<?, ?B/s]

100%|████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 38657.18it/s]


dedup-parliament-10-0.zip:   0%|          | 0.00/153k [00:00<?, ?B/s]